In [2]:
import numpy as np
import matplotlib.pyplot as plt
import GPy
%matplotlib inline
np.set_printoptions(suppress=True, precision=10)
from paramz.transformations import Logexp
import itertools

class SimpleModel(GPy.Model):
    def __init__(self, name, dims, priors=False):
        super(SimpleModel, self).__init__(name)
        self.params = []
        self.peak_loc = range(1,dims+1,1)
        for i,pos in enumerate(self.peak_loc):
            if priors:
                p = GPy.Param('param%d' % i, 1.0, Logexp())
            else:
                p = GPy.Param('param%d' % i, 1.0)
            self.params.append(p)
            self.link_parameter(p)
    
    def log_likelihood(self):
        like = 0
        for i,pos in enumerate(self.peak_loc):
            like -= ((self.params[i])-pos)**2
        return like[0]
    
    def parameters_changed(self):
        for i,pos in enumerate(self.peak_loc):
            self.params[i].gradient = -2*((self.params[i])-pos)
            
def find_likes(m,stepsize=0.3,rangemin=-2,rangemax=7):
    """
    Numerical grid integral over model parameters
    This function returns the sum of the objective
    """
    m.optimize()
    params = m.optimizer_array[None,:].copy()
    param_ranges = []
    for param in params[0]:
        param_ranges.append(np.arange(rangemin,rangemax,stepsize))        
    combs = itertools.product(*param_ranges)
    lsum = 0
    for el in combs:
        lsum+=np.exp(-m._objective(el))
    return lsum*(stepsize**len(params[0]))

def monte_carlo_int(m,steps=1500):
    """
    My quick and dirty monte carlo integration.
    Returns a mean and the bounds of the 95%
    confidence interval.
    """
    m.optimize()
    opt = m.optimizer_array[None,:].copy()
    nparams = len(m.optimizer_array)
    searchwidth = 8
    V = searchwidth**nparams
    randnos = (np.random.rand(steps,nparams)-0.5)*searchwidth
    #randnos[:,2]=0
    test_params = randnos+opt.repeat(steps,0)[:]
    tot = []
    for i in range(steps):
        tot.append(np.exp(-m._objective(test_params[i,:])))
    mean = V*np.mean(tot)
    ste = V*np.std(tot)/np.sqrt(len(tot))
    return mean,mean-ste*1.96,mean+ste*1.96,nparams

warning in stationary: failed to import cython module: falling back to numpy


In [12]:
for priors in [False,True]:
    print("====Simple model",end='')
    if priors:
        print("(with priors)====")
    else:
        print("(no priors)====")
    for dims in range(1,4):
        print("==%d dimensions==" % dims)
        stepsize=0.4*dims #make step size bigger as dims goes up so this isn't too slow
        m2 = SimpleModel('simple',dims,priors)
        numsum = find_likes(m2,stepsize)
        m2.optimize()
        hes = m2.numerical_parameter_hessian()
        m2.optimize()
        #hessum = np.exp(m2.log_likelihood())*1/np.sqrt(np.linalg.det(1/(2*np.pi)*hes))
        hessum = np.exp(-m2._objective(m2.param_array))*1/np.sqrt(np.linalg.det(1/(2*np.pi)*hes))
        ccdpos,ccdres,scalings,z = m2.CCD()
        tot = 0
        print("------------")
        for i, res in enumerate(ccdres):
            m2.param_array[:] = ccdpos[i,:]
            
            tot += res * np.exp(m2.log_likelihood())
            print(res, m2.param_array[:], -m2.log_likelihood())
            #tot+= res*np.exp(-m2._objective(ccdpos[i,:]))
        print("------------")
        print("Weighted CCD approximation")
        print(tot)
        print("Numerical approximation")
        print(numsum)
        print("laplace approximation")
        print(hessum)
        print("Monte carlo")
        print(monte_carlo_int(m2))
        #assert np.isclose(hessum,numsum,rtol=0.3), "Simple Model: Laplace approximation using numerical_parameter_hessian()=%0.4f not equal to numerical grid sum=%0.4f" % (hessum,numsum)

    
print("====GP model====")
#Test numerical_parameter_hessian gives us the right integral for a more complex GP model
#sample data
X = np.arange(0,40,1)[:,None]
Y = np.sin(X/5)+np.random.randn(X.shape[0],X.shape[1])*0.1
k = GPy.kern.RBF(1)

#create model and optimise
m2 = GPy.models.GPRegression(X,Y,k)
m2.Gaussian_noise.fix(0.5)
m2.optimize()

m2.numerical_parameter_hessian()
dims = 2 #equals the number of unfixed parameters
stepsize=0.3
numsum = find_likes(m2,stepsize,rangemin=0.0001,rangemax=20)
m2.optimize()
hes = m2.numerical_parameter_hessian()
hessum = np.exp(m2.log_likelihood())*1/np.sqrt(np.linalg.det(1/(2*np.pi)*hes))
#assert np.isclose(hessum,numsum,rtol=0.1,atol=0), "GP Model: Laplace approximation using numerical_parameter_hessian()=%0.4f not equal to numerical grid sum=%0.4f" % (hessum,numsum)

m2.optimize()
ccdpos,ccdres,scalings,z = m2.CCD()
tot = 0
for i, res in enumerate(ccdres):
    tot+= res*np.exp(-m2._objective(ccdpos[i,:]))
    #m2.param_array[:] = ccdpos[i,:]
    #tot += res * np.exp(m2.log_likelihood())
print("Weighted sum")
print(tot)
print("Laplace")
print(hessum)
print("Numerical approx")
print(numsum)
print("Monte Carlo")
print(monte_carlo_int(m2))

====Simple model(no priors)====
==1 dimensions==
------------
0.173553719008 [ 1.] -0.0
0.206611570248 [ 1.7778174593] 0.605
0.206611570248 [ 0.2221825407] 0.605
0.206611570248 [ 1.7778174593] 0.605
0.206611570248 [ 0.2221825407] 0.605
------------
Weighted CCD approximation
0.624854898049
Numerical approximation
1.77244981163
laplace approximation
1.77245385091
Monte carlo
(1.7671687147824098, 1.6331300740372363, 1.9012073555275832, 1)
==2 dimensions==
------------
0.173553719008 [ 1.  2.] -0.0
0.103305785124 [ 1.7778174593  2.7778174593] 1.21
0.103305785124 [ 0.2221825407  2.7778174593] 1.21
0.103305785124 [ 1.7778174593  1.2221825407] 1.21
0.103305785124 [ 0.2221825407  1.2221825407] 1.21
0.103305785124 [ 2.1  2. ] 1.21
0.103305785124 [-0.1  2. ] 1.21
0.103305785124 [ 1.   3.1] 1.21
0.103305785124 [ 1.   0.9] 1.21
------------
Weighted CCD approximation
0.419997751595
Numerical approximation
3.14159315445
laplace approximation
3.14159265359
Monte carlo
(3.3125096907075533, 2.8002477